# Preparing network file for HuRI data.
HuRI data at `data/networks/species_9606/huri/_raw/HI-union.tsv` downloaded from http://www.interactome-atlas.org/data/HI-union.tsv on June 18, 2020.  

In [3]:
import pandas as pd

In [42]:
mapping_df = pd.read_csv("_raw/results.txt", delimiter="\t")

In [43]:
mapping_df = pd.read_csv("_raw/mapping.txt", delimiter="\t")
mapping_df = mapping_df.dropna(subset=["Ensembl gene ID", "NCBI Gene ID"])
mapping_df["ensembl_id"] = mapping_df["Ensembl gene ID"]
mapping_df["entrez_id"] = mapping_df["NCBI Gene ID"].astype(int)
mapping_df = mapping_df[["ensembl_id", "entrez_id"]]
mapping_df.to_csv("entrez_to_ensembl.csv", index=False)

In [44]:
network_df = pd.read_csv("_raw/HI-union.tsv", delimiter="\t")

In [45]:
network_df["entrez_a"] = network_df.merge(mapping_df, how="left", left_on="ensembl_a", right_on="ensembl_id")["entrez_id"]
network_df["entrez_b"] = network_df.merge(mapping_df, how="left", left_on="ensembl_b", right_on="ensembl_id")["entrez_id"]

In [51]:
df = network_df[["entrez_a", "entrez_b"]].dropna().astype(int)
n_dropped = len(network_df) - len(df)
print(f"Dropped {n_dropped} interactions because we were missing an entrez ID for one or both of the genes.")
network_df = df

Dropped 1165 interactions because we were missing an entrez ID for one or both of the genes.


In [54]:
network_df.to_csv("network.txt", header=False, index=False, sep=' ')